In [1]:
import datetime
import scipy.stats as stats
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
matplotlib.use('Agg')
%matplotlib inline
from dateutil.relativedelta import relativedelta

from finrl import config
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline, get_baseline_tdx,convert_daily_return_to_pyfolio_ts

import torch
# import plotly.express as px

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import EfficientFrontier
from pypfopt import expected_returns
from pypfopt import objective_functions
from pyfolio import timeseries

import plotly.graph_objects as go
from plotly.subplots import make_subplots

########
import backtrader as bt
import backtrader.indicators as btind
import backtrader.analyzers as btanalyzers

import pyfolio as pf
from matplotlib import rcParams

ModuleNotFoundError: No module named 'finrl'

In [2]:
from lutils.stock import LTdxHq

In [3]:
train_start = '2019-01-01'
train_end = '2022-11-13'
trade_start = train_end
trade_end = '2022-11-22'

In [53]:
stock_count = 8

In [5]:
# ddf = pd.read_pickle('d:/ddf.pkl').dropna()

ddf = pd.read_pickle('d:/ddf_1127.pkl').dropna()
# ddf = pd.read_pickle('d:/ddf_0606.pkl').dropna()

In [6]:
ddf = ddf.sort_values(['date','tic'], ignore_index=True)
ddf.index = ddf.date.factorize()[0]

In [7]:
days = ddf.date.unique()
days.shape

(1190,)

In [1]:
days

NameError: name 'days' is not defined

In [8]:
dl = ddf[(ddf['date'] > days[-63]) & (ddf['date'] < trade_start)]
dd = dl.pivot_table(index = 'date',columns = 'tic', values = 'close').pct_change().dropna()
corr = dd.cov().corr()
# corr.style.background_gradient(cmap='coolwarm')

# c1 = corr.abs().unstack().sort_values(ascending = True)
# c1 = corr.unstack().sort_values(ascending = True)

In [9]:
dd

tic,000001,000002,000004,000005,000008,000009,000010,000011,000012,000014,...,603985,603987,603988,603989,603990,603991,603993,603997,603998,603999
date,,,,,,,,,,,,,,,,,,,,,
2022-08-25,0.017742,0.012148,0.027484,-0.005435,0.008230,0.011417,0.022989,0.014808,-0.010753,0.002401,...,-0.060892,-0.023837,-0.037098,0.017845,-0.030923,-0.009493,0.013592,-0.017170,-0.005042,0.003810
2022-08-26,-0.000792,0.000000,-0.037037,0.000000,-0.016327,0.003984,-0.025281,-0.009728,0.015528,-0.010778,...,-0.045423,0.035736,-0.019692,-0.006875,-0.017978,0.042309,-0.005747,0.041230,0.000000,0.022770
2022-08-29,-0.015067,-0.016425,-0.001068,0.005464,0.004149,-0.003968,0.000000,-0.004912,-0.004587,0.006053,...,0.029786,-0.033928,0.028821,-0.000692,0.042563,0.071316,-0.009634,-0.011409,0.010135,0.007421
2022-08-30,0.004831,0.020552,0.018182,0.010870,0.008264,-0.005976,0.031700,0.004936,-0.009217,0.008424,...,-0.045150,-0.006548,-0.021222,-0.047454,0.001317,-0.012351,-0.009728,-0.070604,0.026756,0.012891
2022-08-31,0.021635,0.046570,-0.018908,-0.016129,-0.012295,-0.014028,-0.013966,-0.014735,0.010853,-0.019093,...,-0.060953,-0.016777,-0.067650,-0.037818,-0.068391,-0.093896,-0.033399,-0.086194,-0.009772,-0.007273
2022-09-01,-0.010980,0.012628,-0.008565,-0.016393,-0.012448,-0.022358,0.005666,0.025922,0.009202,0.099757,...,0.021637,0.008531,-0.013953,-0.023054,-0.027294,0.001170,0.002033,-0.013589,-0.003289,-0.003663
2022-09-02,-0.007930,-0.002375,0.014039,0.005556,0.008403,0.045738,0.005634,-0.014577,-0.013678,-0.019912,...,-0.014247,-0.006647,0.028302,0.006190,0.023222,-0.002336,-0.006085,0.099676,0.011551,0.027574
2022-09-05,0.004796,0.022024,0.002130,0.022099,0.025000,-0.007952,0.030812,0.008876,-0.003082,0.006772,...,0.029687,0.002433,-0.011009,-0.023453,-0.016076,-0.021077,0.010204,-0.056743,0.009788,-0.016100
2022-09-06,-0.005569,0.039604,-0.001063,0.010811,0.008130,0.006680,-0.002717,0.024438,0.049459,0.039238,...,0.020106,-0.005461,0.023191,0.048425,0.053820,-0.018660,0.020202,0.032812,0.004847,-0.009091


In [36]:
dd

tic,000002,000004,000005,000007,000009,000012,000019,000020,000021,000023,...,688739,688766,688772,688776,688777,688778,688779,688786,688787,688789
date,,,,,,,,,,,,,,,,,,,,,
2022-02-24,-0.020731,-0.040000,-0.040000,-0.004532,-0.025892,-0.002336,0.033333,-0.084305,-0.023690,-0.037121,...,-0.025615,-0.049478,-0.046341,-0.033293,-0.023914,-0.040458,-0.023490,-0.004444,-0.039484,-0.027045
2022-02-25,-0.015625,0.054748,0.018519,0.000000,0.005747,-0.022248,-0.025806,-0.030362,0.006618,0.011802,...,0.001494,0.011694,0.009974,0.018814,0.061456,0.041304,0.018164,0.001786,0.012956,-0.004315


In [11]:
corr

tic,000001,000002,000004,000005,000008,000009,000010,000011,000012,000014,...,603985,603987,603988,603989,603990,603991,603993,603997,603998,603999
tic,,,,,,,,,,,,,,,,,,,,,
000001,1.000000,0.850133,0.230916,0.221008,0.054555,-0.145954,0.433517,0.650205,0.095252,0.629421,...,-0.543362,0.341797,-0.421365,-0.085897,-0.491459,-0.260974,0.341366,-0.378322,0.036828,0.257055
000002,0.850133,1.000000,-0.069059,-0.029589,-0.058657,-0.385106,0.169000,0.510733,-0.079510,0.646615,...,-0.508572,0.165359,-0.565344,-0.144280,-0.620799,-0.372896,0.218145,-0.412334,-0.152270,-0.092967
000004,0.230916,-0.069059,1.000000,0.602129,0.405654,0.308864,0.614630,0.457799,0.205233,0.261372,...,-0.082837,0.366995,0.107916,0.139936,0.131470,0.190318,0.137845,-0.115286,0.466511,0.704944
000005,0.221008,-0.029589,0.602129,1.000000,0.622673,0.515252,0.776764,0.572135,0.422787,0.223717,...,0.081085,0.357422,0.366394,0.192659,0.308829,0.382219,0.439818,0.069531,0.467346,0.674217
000008,0.054555,-0.058657,0.405654,0.622673,1.000000,0.643900,0.609892,0.317751,0.597215,0.027574,...,0.416153,0.381226,0.581384,0.245810,0.534292,0.348455,0.467141,0.400273,0.286180,0.360562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603991,-0.260974,-0.372896,0.190318,0.382219,0.348455,0.555263,0.218665,0.029398,0.360069,-0.100243,...,0.487745,0.469509,0.608855,0.565361,0.627199,1.000000,0.412452,0.521163,0.483575,0.340618
603993,0.341366,0.218145,0.137845,0.439818,0.467141,0.517950,0.470140,0.405167,0.621556,0.247032,...,0.216376,0.446564,0.458053,0.512524,0.327366,0.412452,1.000000,0.504927,0.097612,0.159424
603997,-0.378322,-0.412334,-0.115286,0.069531,0.400273,0.648579,0.054050,-0.194706,0.586598,-0.287342,...,0.746753,0.187394,0.832363,0.577650,0.801338,0.521163,0.504927,1.000000,-0.018606,0.008666


In [12]:
# corr.style.background_gradient(cmap='coolwarm')

In [13]:
# corr.unstack().sort_values().index.get_level_values(0)[:10]

In [14]:
corr.sum().sort_values() # ascending=False

tic
601288    -786.948491
601398    -733.569122
601939    -688.925879
002541    -657.358337
601988    -616.878697
             ...     
300067    1534.498786
300405    1536.302348
002367    1538.242585
603686    1541.522359
300243    1554.217592
Length: 2790, dtype: float64

In [15]:
# stock_codes = list(np.random.choice(ddf.tic.unique(), 6))

In [16]:
# corr.unstack().sort_values().index.get_level_values(1)

In [17]:
# corr.style.background_gradient(cmap='coolwarm')

In [54]:
_stock_codes = list(corr.unstack().sort_values().index.get_level_values(0)[:stock_count * 2])
stock_codes = []
while len(stock_codes) < stock_count:
    sc = _stock_codes.pop(0)
    if sc not in stock_codes:
        stock_codes.append(sc)

# ['601398', '000738']
# ['601988', '603538'] # 2022-02-08
# ['600377', '000887'] # 2022-02-14
# ['301040', '000573'] # 2022-02-15
# ['301040', '000573', '300384'] # 2022-02-20

In [19]:
# stock_codes = list(corr.sum().sort_values().index.values[:2]) # ascending=False
# ['000921', '600365', '000609', '601868', '002372', '001965']
# ['601868', '000501', '603789', '002311', '002873', '000921']
# ['000921', '603789', '002372', '601868', '300675', '002032']
# ['600775', '603789', '300622', '603228', '600172', '001965']

In [20]:
# stock_codes = ['601186', '002645', '600546', '688222', '600107', '603028']
# stock_codes = ['601988', '603538']

# ['603003', '603326', '002533', '600459']

In [21]:
stock_codes

['300516', '601288', '002698', '300183']

In [55]:
stock_codes

['300516',
 '601288',
 '002698',
 '300183',
 '601988',
 '300570',
 '000534',
 '601939']

In [22]:
# stock_codes = ['000921', '002032', '300406', '603789']
# stock_codes = ['000609', '000921', '001965', '002372']

In [23]:
ltdxhq = LTdxHq()

indexs = None
dfs = []
for code in stock_codes:
    df = ltdxhq.get_k_data_daily(code, start='2018-01-01') # 2014-01-01
    
    if indexs is None:
        indexs = df.index
    else:
        indexs = indexs.union(df.index)
    
#     df = df.assign(date = df.index)
#     df = df.assign(day = df.index.weekday)
#     df.date = df.date.dt.strftime('%Y-%m-%d')
    df = df.assign(tic = code)
#     df.index = range(df.shape[0])
    
    dfs.append(df)
    print('----------- over %s min: %s max: %s -----------' % (code, df.index.min(), df.index.max()))

for i, df in enumerate(dfs):
    df = df.reindex(indexs)
    df = df.assign(date = df.index)
    df = df.assign(day = df.index.weekday)
    df.index = range(df.shape[0])
    
    dfs[i] = df.ffill()
    
# df = pd.concat(dfs)
# df.index = range(df.shape[0])

ltdxhq.close()

----------- over 300516 min: 2018-01-02 00:00:00 max: 2022-11-25 00:00:00 -----------
----------- over 601288 min: 2018-01-02 00:00:00 max: 2022-11-25 00:00:00 -----------
----------- over 002698 min: 2018-01-02 00:00:00 max: 2022-11-25 00:00:00 -----------
----------- over 300183 min: 2018-01-02 00:00:00 max: 2022-11-25 00:00:00 -----------


In [24]:
df = pd.concat(dfs)

In [25]:
%matplotlib inline

In [26]:
# hist = df[df.tic == '601868 '].close.hist() # bins=3)

# df[['open', 'close', 'high', 'low']].plot.hist(bins=100, alpha=0.5)
# df[df.tic == '000921'].close.pct_change().hist(bins=100)
# sns.distplot(df[df.tic == '000921'].close)

In [27]:
# sns.distplot(df[df.tic == '000921'].close.pct_change())

In [28]:
# stat, p = shapiro(df[df.tic == '000921'].close.pct_change())
# stat, p

In [29]:
df

,open,close,high,low,volume,amount,tic,date,day
0,29.82,29.92,29.94,29.55,311300.0,14106950.0,300516,2018-01-02,1
1,29.88,30.09,30.20,29.71,308938.0,14126594.0,300516,2018-01-03,2
2,30.07,29.54,30.07,29.50,388750.0,17598828.0,300516,2018-01-04,3
3,29.63,29.70,29.82,29.29,330300.0,14894724.0,300516,2018-01-05,4
4,29.77,29.61,29.77,29.34,330300.0,14821094.0,300516,2018-01-08,0
...,...,...,...,...,...,...,...,...,...
1185,13.92,14.05,14.13,13.85,2357281.0,33029980.0,300183,2022-11-21,0
1186,14.04,13.82,14.08,13.75,2626381.0,36554276.0,300183,2022-11-22,1
1187,13.78,13.57,13.83,13.30,3726462.0,50390360.0,300183,2022-11-23,2
1188,13.62,13.49,13.77,13.41,1945967.0,26368242.0,300183,2022-11-24,3


In [30]:
fe = FeatureEngineer(use_technical_indicator=True,
                     use_turbulence=False,
                     user_defined_feature = False)

df = fe.preprocess_data(df)

Successfully added technical indicators


In [31]:
# add covariance matrix as states
df=df.sort_values(['date','tic'], ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback, len(df.index.unique())):
    data_lookback = df.loc[i-lookback:i, :]
    price_lookback = data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values
    cov_list.append(covs)


df_cov = pd.DataFrame({'date': df.date.unique()[lookback:], 'cov_list': cov_list,'return_list': return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date', 'tic']).reset_index(drop = True)

In [32]:
df['daily_variance'] = (df.high - df.low) / df.close

In [33]:
dl = df.loc[-5:, :]
dd = dl.pivot_table(index = 'date',columns = 'tic', values = 'close').pct_change().dropna()
dd.cov().corr().style.background_gradient(cmap='coolwarm')

tic,002698,300183,300516,601288
tic,,,,
002698,1.000000,0.346540,-0.141640,-0.658190
300183,0.346540,1.000000,0.023280,-0.718684
300516,-0.141640,0.023280,1.000000,-0.481559
601288,-0.658190,-0.718684,-0.481559,1.000000


In [34]:
train = data_split(df, train_start, train_end) # 2021-07-01 2022-01-01

In [35]:
train.head()

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list,daily_variance
0,6.20,6.26,6.30,6.15,3272837.0,32593290.0,002698,2019-01-15,1,0.054580,6.237360,5.402640,57.031312,121.074741,38.902549,5.906000,5.876000,"[[0.0007080953382086647, 0.0004031987116246608...",tic 002698 300183 300516 60...,0.023962
0,11.43,11.77,11.90,11.40,5868258.0,71960336.0,300183,2019-01-15,1,-0.029497,11.891502,11.012498,50.396977,-7.799274,4.949417,11.721667,11.403000,"[[0.0007080953382086647, 0.0004031987116246608...",tic 002698 300183 300516 60...,0.042481
0,19.46,19.75,19.86,19.46,413273.0,12377165.0,300516,2019-01-15,1,0.012271,19.935345,17.881655,50.492091,39.382862,5.533352,19.286000,19.459333,"[[0.0007080953382086647, 0.0004031987116246608...",tic 002698 300183 300516 60...,0.020253
0,2.78,2.81,2.81,2.78,173063104.0,608349376.0,601288,2019-01-15,1,-0.015444,2.873275,2.768725,46.385209,-90.000000,7.113922,2.830667,2.892000,"[[0.0007080953382086647, 0.0004031987116246608...",tic 002698 300183 300516 60...,0.010676
1,6.30,6.25,6.30,6.16,2726153.0,27088120.0,002698,2019-01-16,2,0.073998,6.300133,5.380867,56.849346,121.457152,38.902549,5.906000,5.890000,"[[0.0007078736608872213, 0.0004030063542824461...",tic 002698 300183 300516 60...,0.022400


In [87]:
train.head()

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list,daily_variance
0,2.78,2.81,2.81,2.78,173063104.0,6.083494e+08,601288,2019-01-15,1,-0.015444,2.873275,2.768725,46.385209,-90.000000,7.113922,2.830667,2.892000,[[0.00023090498003643345]],tic 601288 date 2018...,0.010676
1,2.80,2.80,2.81,2.78,148983632.0,5.233144e+08,601288,2019-01-16,2,-0.015456,2.869706,2.767294,45.741203,-93.877551,7.113922,2.828000,2.887167,[[0.00023076367842540877]],tic 601288 date 2018...,0.010714
2,2.81,2.81,2.82,2.80,249369776.0,8.815909e+08,601288,2019-01-17,3,-0.014492,2.869340,2.766660,46.509467,-48.936170,2.739529,2.826000,2.883500,[[0.00023077203275489368]],tic 601288 date 2018...,0.007117
3,2.82,2.85,2.86,2.81,353170336.0,1.260595e+09,601288,2019-01-18,4,-0.010380,2.873114,2.766886,49.470016,51.197053,12.533862,2.825667,2.879667,[[0.00023158196775813138]],tic 601288 date 2018...,0.017544
4,2.85,2.85,2.89,2.85,291304544.0,1.047007e+09,601288,2019-01-21,0,-0.007040,2.875756,2.770244,49.470016,122.281640,21.848389,2.826000,2.875667,[[0.00023158196775813135]],tic 601288 date 2018...,0.014035


In [36]:
train[train['cci_30'] == np.inf]

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list,daily_variance


In [37]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A portfolio allocation environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self,
                 df,
                 stock_dim,
                 hmax,
                 initial_amount,
                 transaction_cost_pct,
                 reward_scaling,
                 state_space,
                 action_space,
                 tech_indicator_list,
                 turbulence_threshold=None,
                 lookback=252,
                 day=0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
#         self.reward = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low=0, high=1,shape=(self.action_space,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        self.turbulence_threshold = turbulence_threshold
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1 / self.stock_dim] * self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique()) - 1

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
#             plt.plot(df.daily_return.cumsum(),'r')
#             plt.savefig('results/cumulative_reward.png')
#             plt.close()
            
#             plt.plot(self.portfolio_return_memory,'r')
#             plt.savefig('results/rewards.png')
#             plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() != 0:
                sharpe = (252 ** 0.5) * df_daily_return['daily_return'].mean() / df_daily_return['daily_return'].std()
                print("Sharpe: ", sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            weights = self.softmax_normalization(actions)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state = np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list], axis=0)
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values) - 1) * weights)
#             log_portfolio_return = np.log(sum((self.data.close.values / last_day_memory.close.values) * weights))
            # update portfolio value
            new_portfolio_value = self.portfolio_value * (1 + portfolio_return)
#             new_portfolio_value = self.portfolio_value * (1 + log_portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value / self.initial_amount

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day, :]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator / denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [38]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f'Stock Dimension: {stock_dimension}, State Space: {state_space}')

Stock Dimension: 4, State Space: 4


In [39]:
train

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list,daily_variance
0,6.20,6.26,6.30,6.15,3272837.0,3.259329e+07,002698,2019-01-15,1,0.054580,6.237360,5.402640,57.031312,121.074741,38.902549,5.906000,5.876000,"[[0.0007080953382086647, 0.0004031987116246608...",tic 002698 300183 300516 60...,0.023962
0,11.43,11.77,11.90,11.40,5868258.0,7.196034e+07,300183,2019-01-15,1,-0.029497,11.891502,11.012498,50.396977,-7.799274,4.949417,11.721667,11.403000,"[[0.0007080953382086647, 0.0004031987116246608...",tic 002698 300183 300516 60...,0.042481
0,19.46,19.75,19.86,19.46,413273.0,1.237716e+07,300516,2019-01-15,1,0.012271,19.935345,17.881655,50.492091,39.382862,5.533352,19.286000,19.459333,"[[0.0007080953382086647, 0.0004031987116246608...",tic 002698 300183 300516 60...,0.020253
0,2.78,2.81,2.81,2.78,173063104.0,6.083494e+08,601288,2019-01-15,1,-0.015444,2.873275,2.768725,46.385209,-90.000000,7.113922,2.830667,2.892000,"[[0.0007080953382086647, 0.0004031987116246608...",tic 002698 300183 300516 60...,0.010676
1,6.30,6.25,6.30,6.16,2726153.0,2.708812e+07,002698,2019-01-16,2,0.073998,6.300133,5.380867,56.849346,121.457152,38.902549,5.906000,5.890000,"[[0.0007078736608872213, 0.0004030063542824461...",tic 002698 300183 300516 60...,0.022400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,2.74,2.76,2.77,2.73,334545504.0,9.201576e+08,601288,2022-11-10,3,-0.027966,2.901524,2.688476,43.236143,-89.190041,26.342384,2.815000,2.825833,"[[0.0003984734813104573, 0.0002236552882936448...",tic 002698 300183 300516 60...,0.014493
927,15.10,14.72,15.31,14.59,8344541.0,1.238431e+08,002698,2022-11-11,4,0.070230,16.286813,14.399187,48.805653,-31.010933,13.297941,15.085333,15.403333,"[[0.0003984056062151577, 0.0002240763701222338...",tic 002698 300183 300516 60...,0.048913
927,13.68,13.41,13.80,13.41,4126703.0,5.611900e+07,300183,2022-11-11,4,0.069273,14.077202,12.944798,47.965653,53.178165,4.912010,13.195333,13.864333,"[[0.0003984056062151577, 0.0002240763701222338...",tic 002698 300183 300516 60...,0.029083
927,27.41,26.86,27.93,26.78,1336850.0,3.637973e+07,300516,2022-11-11,4,0.688204,28.959932,23.158068,52.814153,93.287998,13.364728,25.198000,26.038833,"[[0.0003984056062151577, 0.0002240763701222338...",tic 002698 300183 300516 60...,0.042815


In [40]:
# ['daily_variance', 'change', 'log_volume', 'close','day', 'macd', 'rsi_30', 'boll_ub', 'dx_30']
# ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'boll_ub', 'dx_30', 'close_30_sma', 'close_60_sma'] # cci_30
tech_indicator_list = ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'daily_variance', 'dx_30', 'close_30_sma', 'close_60_sma']
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 10000 * stock_dimension, 
    "transaction_cost_pct": 0, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": tech_indicator_list, # config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-1
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

In [41]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [42]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    'n_steps': 2048,
    'ent_coef': 0.005,
    'learning_rate': 0.0001,
    'batch_size': 128,
}
model_ppo = agent.get_model('ppo', model_kwargs = PPO_PARAMS, tensorboard_log='D:/code/python/logs')

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
Using cuda device


In [43]:
agent.train_model(model=model_ppo, tb_log_name='ppo', total_timesteps=20000)

d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

Logging to D:/code/python/logs\ppo_10
begin_total_asset:40000
end_total_asset:74189.3293009158
Sharpe:  0.7868196823107058
begin_total_asset:40000
end_total_asset:62163.75784232267
Sharpe:  0.5841506969264982
----------------------------------
| time/              |           |
|    fps             | 244       |
|    iterations      | 1         |
|    time_elapsed    | 8         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 1.4481622 |
----------------------------------
begin_total_asset:40000
end_total_asset:67393.00329499599
Sharpe:  0.680474139569359
begin_total_asset:40000
end_total_asset:70027.00036027217
Sharpe:  0.7078710240833184
-----------------------------------------
| time/                   |             |
|    fps                  | 322         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx

begin_total_asset:40000
end_total_asset:66586.62173887783
Sharpe:  0.6606908769523694
begin_total_asset:40000
end_total_asset:79135.36383017678
Sharpe:  0.8430601380883118
-----------------------------------------
| time/                   |             |
|    fps                  | 432         |
|    iterations           | 9           |
|    time_elapsed         | 42          |
|    total_timesteps      | 18432       |
| train/                  |             |
|    approx_kl            | 0.004200201 |
|    clip_fraction        | 0.0147      |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.69       |
|    explained_variance   | -0.00387    |
|    learning_rate        | 0.0001      |
|    loss                 | 209         |
|    n_updates            | 80          |
|    policy_gradient_loss | -0.00861    |
|    reward               | 1.3355047   |
|    std                  | 1           |
|    value_loss           | 442         |
------------------------------

In [44]:
trade = data_split(df, trade_start, trade_end) # '2021-12-20', '2023-01-01' trade_start, trade_end
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)

In [45]:
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [46]:
df_daily_return_ppo, df_actions_ppo = DRLAgent.DRL_prediction(model=model_ppo, environment=e_trade_gym)
time_ind = pd.Series(df_daily_return_ppo.date)
ppo_cumpod =(df_daily_return_ppo.daily_return + 1).cumprod() - 1
DRL_strat_ppo = convert_daily_return_to_pyfolio_ts(df_daily_return_ppo)

perf_func = timeseries.perf_stats 

perf_stats_all_ppo = perf_func(returns=DRL_strat_ppo, 
                               factor_returns=DRL_strat_ppo, 
                               positions=None, transactions=None, turnover_denom='AGB')

begin_total_asset:40000
end_total_asset:41789.67216834092
Sharpe:  13.498681000017015
hit end!


d:\soft\python\python37\lib\site-packages\empyrical\stats.py:799: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)


In [47]:
e_trade_gym.actions_memory

[[0.25, 0.25, 0.25, 0.25]]

In [ ]:
e_trade_gym.portfolio_return_memory

In [ ]:
e_trade_gym.portfolio_value

In [ ]:
df_daily_return_ppo

In [43]:
DRLAgent?

In [48]:
class StrategyText(bt.Strategy):
    
#     params = (('period', 5),)
    
    def __init__(self):
#         self.sma_turnover_rate = btind.SMA(self.data0.turnover_rate, period=self.p.period, plot=False)
#         self.sma_turnover_rate15 = btind.SMA(self.data0.turnover_rate, period=7, plot=False)
#         self.crossover = btind.CrossOver(self.sma_turnover_rate, self.sma_turnover_rate15)


    def next(self):
#         if not self.position:
#             if self.crossover > 0:
#                 self.buy()
#         elif not self.crossover < 0:
#             self.close()
        
#         if not self.position:
#             if self.data0.close > self.sma_turnover_rate:
#                 self.buy()
#         elif self.data0.close < self.sma_turnover_rate:
#             self.close()

        if not self.position:
            if self.data0.turnover_rate / self.sma_turnover_rate[-1] > 2.5 and self.data0.change_rate < -4:
                self.buy()
        elif self.data0.turnover_rate / self.sma_turnover_rate[-1] < 1.5:
            self.close()
            
        

IndentationError: expected an indented block (<ipython-input-48-baac3efd033e>, line 11)

In [38]:
# import matplotlib.pyplot as plt
# import squarify 
# %matplotlib inline
# sizes=[40, 30, 5, 25]
# label=["A", "B", "C", "D"]
# squarify.plot(sizes=sizes, label=label, alpha=0.6)
# plt.show()

In [61]:
# stocks_test = ['002533', '600459', '603003', '603326']
# pers = [0.25, 0.25, 0.25, 0.25]

# stocks_test = ['300652', '600900', '601398']
# pers = [.3333, .3333, .3333]

# stocks_test = ['300516', '601288', '002698', '300183'] # ['300395', '600000', '601988', '603040']
stocks_test = ['300516', '601288', '002698', '300183', '601988', ] # '300570', ] #'000534', '601939']
# pers = [0.25, 0.25, 0.25, 0.25]
pers = [1 / len(stocks_test)] * len(stocks_test)


ltdxhq = LTdxHq()

indexs = None
dfs = []
buy_money = 5000 * len(stocks_test)
left_money = 0
after_money = 0
keeps = []
for code, per in zip(stocks_test, pers):
    df = ltdxhq.get_k_data_daily(code, start=trade_start) #, end=trade_end) # 2014-01-01
    keep = (buy_money * per / (df.iloc[0].close * 100))
    keeps.append(keep)
    
    left_money += buy_money * per - keep * df.iloc[0].close * 100
    
    after_money += df.iloc[-1].close * 100 * keep
    
after_money += left_money
    

In [62]:
after_money

25406.759255208737